In [0]:
qr = f"""
with tr_mac as (
  select 
    heds1072.mac_addr as mac_addr, 
    rbt.mac_addr as real_mac
  from sandbox.z_luckyminki_choi.heds1072 heds1072
  inner join eic_data_private.tlamp.rbt_mac_addr_webos60 rbt
  on heds1072.mac_addr = rbt.mac_addr_hashed
), actv as (
  select 
    mac_addr,
    min(crt_date) as min_crt_date,
    max(last_chg_date) as max_last_chg_date
  from eic_data_ods.tlamp.activation_date
  where 1=1
  and mac_addr in (select mac_addr from tr_mac)
  group by mac_addr
), normal as (
  select 
    mac_addr,
    max(log_create_time) as max_log_create_time,
    max(accum_run_time) as max_accum_run_time
  from (
    select 
      mac_addr,
      log_create_time,
      accum_run_time
    from eic_data_ods.tlamp.normal_log_webos60
    where 1=1
    and mac_addr in (select mac_addr from tr_mac)
    union all
    select 
      mac_addr,
      log_create_time,
      accum_run_time
    from eic_data_ods.tlamp.lginput2_webos60
    where 1=1
    and mac_addr in (select mac_addr from tr_mac)
    )
  group by mac_addr
)
select 
  tr_mac.real_mac,
  org.*,
  actv.min_crt_date,
  actv.max_last_chg_date,
  normal.max_log_create_time,
  normal.max_accum_run_time
from sandbox.z_luckyminki_choi.heds1072 org
left outer join tr_mac
on org.mac_addr=tr_mac.mac_addr
left outer join actv
on org.mac_addr=actv.mac_addr
left outer join normal
on org.mac_addr=normal.mac_addr
"""

df= spark.sql(qr)

df.coalesce(1).write.format("com.databricks.spark.csv").mode("overwrite").option("header", "true").save('s3://s3-lge-he-inbound-eic-dev/HEDS/HEDS-1072-1')